In [1]:
# import Dependencies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

In [2]:
def generating_browser():
    chrome_options = webdriver.ChromeOptions()
    set_arg = chrome_options.add_argument
    set_experiment = chrome_options.add_experimental_option
    chrome_options.add_argument("--remote-allow-origins=*")

    # set options to make browser easier
    set_arg('disable-infobars') # disable infobar popus
    # set_arg('start-maximized') # start selenium in a maximized window
    set_arg('disable-dev-shm-usage') # disable developer mode error for linux and mac only
    set_arg('no-sandbox') # disable security sandbox layer **important
    set_arg('disable-blink-features=AutomationControlled') # disable detection of automation test
    set_experiment('excludeSwitches', ['enable-automation']) # for allowing the automation

    # disable password saving popup and location sharing
    prefs = {"credentials_enable_service": False,
     "profile.password_manager_enabled": False,
     "profile.default_content_setting_values.geolocation" :2}
    set_experiment("prefs", prefs)

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=chrome_options)
    return driver

In [3]:
url = "https://www.youtube.com/@5MinuteCraftsYouTube/videos"

In [4]:
fox_driver = generating_browser()
fox_driver.get(url)
fox_driver.maximize_window()

### Content selector

grid_selector = `div#meta.ytd-rich-grid-media`

vid_link = `ytd-rich-grid-media.style-scope.ytd-rich-item-renderer div ytd-thumbnail a`

vid_heading = `ytd-rich-grid-media.style-scope.ytd-rich-item-renderer div#details h3`

vid_views =`//div[@id = "metadata-line"]//span[1]`

vid_upload_time =`//div[@id = "metadata-line"]//span[2]`



In [5]:
def pageLoader():
    # Scrolling down to the whole plage
    scroll_Pause_time = 1
    # Get page scroll height
    last_height = fox_driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        fox_driver.execute_script("window.scrollTo(0, arguments[0]);", last_height)
        # wait to load the page
        sleep(10)
        # compare new scroll height to last scroll height to break the loop
        new_height = fox_driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    loading_report = 'Page Loading Done, starting to scrape data'
    print(loading_report)

In [6]:

videos_details = []
def getData():

   grid_selector = fox_driver.find_elements(By.CSS_SELECTOR, "div#meta.ytd-rich-grid-media")
   for video in grid_selector:
      video_heading = video.find_element(By.CSS_SELECTOR, "h3 a yt-formatted-string").text
      video_views = video.find_element(By.CSS_SELECTOR, "div#metadata-line > span:nth-child(3)").text
      video_uploaded_time = video.find_element(By.CSS_SELECTOR, "div#metadata-line > span:nth-child(4)").text
      video_link = video.find_element(By.CSS_SELECTOR, "h3 a").get_attribute("href")
      video_info ={
         'Video Heading': video_heading,
         'Total Views': video_views,
         'Video Uploaded':  video_uploaded_time,
         'Video Link': video_link
      }
      videos_details.append(video_info)




In [7]:
pageLoader()

Page Loading Done, starting to scrape data


In [8]:
getData()
len(videos_details)

5914

In [9]:
videos_details[0]

{'Video Heading': 'PRINCESS GETS READY FOR SCHOOL ✨ Magical DIY Ideas For Parents',
 'Total Views': '12K views',
 'Video Uploaded': '2 hours ago',
 'Video Link': 'https://www.youtube.com/watch?v=f95bJYP3Qew'}

In [10]:
import csv

def saveToCSV():
    file_name = input("Enter you file name: ")
    with open(f"./data/{file_name}.csv", mode="a", encoding="utf-8") as file:
        field_names = ["Video Heading", "Total Views", "Video Uploaded", "Video Link"]
        csv_writer = csv.DictWriter(file, fieldnames=field_names)
        csv_writer.writeheader()
        for video in videos_details:
            csv_writer.writerow(video)
    print('Your Datas Have been Extracted to csv.')


In [11]:
saveToCSV()

Your Datas Have been Extracted to csv.


In [ ]:
quit()